In [62]:
#Importing data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
# Define the models

models = [
    ('Lasso Regression', Lasso()),
    ('Random Forest', RandomForestRegressor()),
    ('XGBoost', XGBRegressor())
]

# models = [
#     ('Linear Regression', LinearRegression()),
#     ('Lasso Regression', Lasso()),
#     ('Ridge Regression', Ridge()),
#     ('SVM', SVR()),
#     ('Random Forest', RandomForestRegressor()),
#     ('Decision Tree', DecisionTreeRegressor()),
#     ('Neural Network', MLPRegressor(max_iter=50000)),
#     ('XGBoost', XGBRegressor())
# ]
#Importing data
#for 99 samples we are providing the metadata file here
metadata = pd.read_csv('metadata_labeled.csv')
metadata = metadata.drop_duplicates(subset='Run', keep='first')
metadata = metadata.drop('Label',axis=1)
#This is multiqc quality data file
multiqc = pd.read_csv('multiqc_data.csv')
#we are passing the labels file here
labels= pd.read_csv('labels_splitted.csv',sep='\t')
#Merging the metadata and multiqc data
all_features = pd.merge(metadata, multiqc, on='Run', how='inner')
print("Enter the Machine you are using, like M1 as 1, M2 as 2, M3 as 3...so on")
input_machine = input()
#extract for M0 from label dataframe along with Run column
label_1= labels.iloc[:,[0,(int(input_machine)*2)-1]]
label_2= labels.iloc[:,[0,int(input_machine)*2]]
print("You have selected Machine", input_machine)
#rename the last column as label
label_1.columns = ['Run','Label']
label_2.columns = ['Run','Label']
#conver the label column as integer
# label_1['Label'] = label_1['Label'].astype(int)
# label_2['Label'] = label_2['Label'].astype(int)
#merging fq2bam labels with all_features
all_features_df1 = pd.merge(all_features, label_1, on='Run', how='inner')
#merging haplotype labels with all_features
all_features_df2 = pd.merge(all_features, label_2, on='Run', how='inner')
# remove columns with no data
all_features_df1 = all_features_df1.dropna(axis=1, how='all')
all_features_df2 = all_features_df2.dropna(axis=1, how='all')
# remove columns wiht only one unique value
all_features_df1 = all_features_df1.loc[:,all_features_df1.apply(pd.Series.nunique) != 1]
all_features_df2 = all_features_df2.loc[:,all_features_df2.apply(pd.Series.nunique) != 1]
# remove columns with more than 40% missing values
all_features_df1 = all_features_df1.loc[:,all_features_df1.isnull().mean() < 0.4]
all_features_df2 = all_features_df2.loc[:,all_features_df2.isnull().mean() < 0.4]
# remove columns with more than 40% zeros
all_features_df1 = all_features_df1.loc[:,(all_features_df1 == 0).mean() < 0.4]
all_features_df2 = all_features_df2.loc[:,(all_features_df2 == 0).mean() < 0.4]
# remove non-numeric columns
all_features_df1 = all_features_df1.select_dtypes(include=[np.number])
all_features_df2 = all_features_df2.select_dtypes(include=[np.number])
#remove all NaN values
#Replace nan with mean of that column
all_features_df1 = all_features_df1.fillna(all_features_df1.mean())
all_features_df2 = all_features_df2.fillna(all_features_df2.mean())
del metadata, multiqc, labels, label_1, label_2, input_machine, all_features

Enter the Machine you are using, like M1 as 1, M2 as 2, M3 as 3...so on
You have selected Machine 5


In [63]:
from sklearn.model_selection import cross_val_score, KFold

def train(X, y):
    # Scale the data
    sc = StandardScaler()
    X = sc.fit_transform(X)

    # Define a function to train a model and calculate metrics using cross-validation
    def train_and_evaluate(model, X, y):
        kf = KFold(n_splits=10, shuffle=True, random_state=0)
        mae_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
        mse_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
        r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')

        mae = -mae_scores.mean()
        mse = -mse_scores.mean()
        r2 = r2_scores.mean()

        try:
            model.fit(X, y)
            feature_importance = model.feature_importances_
        except AttributeError:
            feature_importance = None

        return mae, mse, r2, feature_importance

    # Define a function to display the metrics in a table
    def display_results(results):
        print(f"{'Model':<20}{'MAE':<10}{'MSE':<10}{'R2':<10}")
        for name, metrics in results.items():
            mae, mse, r2, feature_importance = metrics
            if feature_importance is not None:
                feature_importance = [f"{value:.2f}" for value in feature_importance]
                feature_importance = ' '.join(feature_importance)
            else:
                feature_importance = ''
            print(f"{name:<20}{mae:<10.2f}{mse:<10.2f}{r2:<10.2f}")

    # Train each model, calculate the metrics, and add the results to the table
    results = {}
    for name, model in models:
        results[name] = train_and_evaluate(model, X, y)

    # Sort the results based on R2 value
    results_sorted = dict(sorted(results.items(), key=lambda item: item[1][2], reverse=True))
    display_results(results_sorted)

In [64]:
# Train the models for stage-I fq2bam
df_all_features = all_features_df1
df_all_features["Label"]

0     1155
1     1149
2     1071
3     1092
4      412
      ... 
93     352
94     263
95     342
96    1224
97     247
Name: Label, Length: 98, dtype: int64

In [65]:
# # CASE 1 WITH 386 FEATURES
# df = df_all_features
# X = df_all_features
# X = X.loc[:,~X.columns.duplicated()]
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# # print("CASE 1 WITH 386 FEATURES")
# # train(X,y)
# # print("Change the model here")
# model = RandomForestRegressor()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# model.fit(X_train, y_train)
# feature_importance = model.feature_importances_
# features = X.columns
# features_importance = dict(zip(features, feature_importance))
# features_importance = dict(sorted(features_importance.items(), key=lambda item: item[1], reverse=True))
# features_importance = dict(list(features_importance.items())[:10])
# features_importance
# # CASE 2 WITH TOP-10 FEATURES
# # keep the features only in top_features from features_importance
# features_importance = list(features_importance.keys())
# top_features = X[features_importance]
# top_features['Label'] = y
# X = top_features
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# # print("CASE 2 WITH TOP-10 FEATURES")
# # train(X,y)
# # CASE 3 WITH SIZE_ONLY FEATURE
# df_size_only = df.loc[:,['size_MB','Label']] #size_only feature
# X = df_size_only
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# print(X.shape)
# print("CASE 3 WITH SIZE_ONLY FEATURE")
# train(X,y)
# # CASE 4 SELECTED FEATURES ONLY
# selected_features = ['size_MB', 'bases', 'InsertSize', 'None_mqc-generalstats-fastqc-total_sequences', 'spots','spots_with_mates','Unique Reads','Duplicate Reads']
# selected_features_df = df[selected_features]
# selected_features_df['Label'] = y
# X = selected_features_df
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# # print("CASE 4 SELECTED FEATURES ONLY")
# # train(X,y)
# # Case 5 with combined features
# #create a column that contains sum of pbsq of all columns prefix with pbsq and then remove all pbsq columns except the sum column
# pbsq_columns = [col for col in df.columns if 'pbsq' in col]
# pbnc_columns = [col for col in df.columns if 'pbnc' in col]
# psgc_columns = [col for col in df.columns if 'psgc' in col]
# psgcp_columns = [col for col in df.columns if 'psgcp' in col]
# pss_columns = [col for col in df.columns if 'pss' in col]
# sdl_columns = [col for col in df.columns if 'sdl' in col]
# df['sum_pbnc'] = df[pbnc_columns].sum(axis=1)
# df['sum_pbsq'] = df[pbsq_columns].sum(axis=1)
# df['sum_psgc'] = df[psgc_columns].sum(axis=1)
# df['sum_psgcp'] = df[psgcp_columns].sum(axis=1)
# df['sum_pss'] = df[pss_columns].sum(axis=1)
# df['sdl'] = df[sdl_columns].sum(axis=1)
# #remove all columns from df where prefix is pbsq
# df = df.loc[:,~df.columns.str.startswith('pbsq')]
# df = df.loc[:,~df.columns.str.startswith('pbnc')]
# df=df.loc[:,~df.columns.str.startswith('psgc')]
# df=df.loc[:,~df.columns.str.startswith('psgcp')]
# df=df.loc[:,~df.columns.str.startswith('pss')]
# df=df.loc[:,~df.columns.str.startswith('sdl')]  
# #add sum_pbsq column to df
# df['Label'] = y
# X = df
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# print("CASE 5 WITH COMBINED FEATURES")
# print(X.shape)
# train(X,y)

#### For fq2bam

In [66]:
# CASE 3 WITH SIZE_ONLY FEATURE
df = df_all_features
X = df_all_features
X = X.loc[:,~X.columns.duplicated()]
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
df_size_only = df.loc[:,['size_MB','Label']] #size_only feature
X = df_size_only
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
print(X.shape)
print("CASE 1 WITH SIZE_ONLY FEATURE")
train(X,y)

(98, 1)
CASE 1 WITH SIZE_ONLY FEATURE


Model               MAE       MSE       R2        
Random Forest       9.70      196.50    0.70      
XGBoost             10.77     242.30    0.63      
Lasso Regression    11.52     237.05    0.60      


In [67]:
# Case 6 SELECTED FEATURES ONLY - selected xx features
# # With features mentioned in the draft paper
df=df_all_features
pbsq_columns = [col for col in df.columns if 'pbsq' in col]
pbnc_columns = [col for col in df.columns if 'pbnc' in col]
psgc_columns = [col for col in df.columns if 'psgc' in col]
psgcp_columns = [col for col in df.columns if 'psgcp' in col]
pss_columns = [col for col in df.columns if 'pss' in col]
sdl_columns = [col for col in df.columns if 'sdl' in col]
df['sum_pbnc'] = df[pbnc_columns].sum(axis=1)
df['sum_pbsq'] = df[pbsq_columns].sum(axis=1)
df['sum_psgc'] = df[psgc_columns].sum(axis=1)
df['sum_psgcp'] = df[psgcp_columns].sum(axis=1)
df['sum_pss'] = df[pss_columns].sum(axis=1)
df['sdl'] = df[sdl_columns].sum(axis=1)
#remove all columns from df where prefix is pbsq
df = df.loc[:,~df.columns.str.startswith('pbsq')]
df = df.loc[:,~df.columns.str.startswith('pbnc')]
df=df.loc[:,~df.columns.str.startswith('psgc')]
df=df.loc[:,~df.columns.str.startswith('psgcp')]
df=df.loc[:,~df.columns.str.startswith('pss')]
df=df.loc[:,~df.columns.str.startswith('sdl')]
df.columns
df_2 = df.loc[:,['spots','bases','avgLength','size_MB','InsertSize','None_mqc-generalstats-fastqc-percent_duplicates',
                 'None_mqc-generalstats-fastqc-percent_gc','None_mqc-generalstats-fastqc-avg_sequence_length',
                 'None_mqc-generalstats-fastqc-total_sequences','Unique Reads','fsch_Per Tile Sequence Quality',
                 'fsch_Per Sequence Quality Scores','fsch_Per Base Sequence Content','fsch_Per Sequence GC Content',
                 'fsch_Per Base N Content', 'fsch_Overrepresented Sequences','fsch_Per Base Sequence Quality',
                 'sum_pbnc','sum_pbsq','sum_pss','sum_psgc'
                 ]]
df_2['Label'] = y
X = df_2
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
print("Case 2 SELECTED FEATURES ONLY")
train(X,y)

Case 2 SELECTED FEATURES ONLY
Model               MAE       MSE       R2        
Random Forest       8.52      139.76    0.76      
XGBoost             9.28      167.45    0.70      
Lasso Regression    11.21     215.48    0.63      


#### For HTVC 

In [68]:
# Train the models for stage-I htvc
df_all_features = all_features_df2

In [69]:
# CASE 3 WITH SIZE_ONLY FEATURE
df = df_all_features
X = df_all_features
X = X.loc[:,~X.columns.duplicated()]
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
df_size_only = df.loc[:,['size_MB','Label']] #size_only feature
X = df_size_only
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
print(X.shape)
print("CASE 1 WITH SIZE_ONLY FEATURE")
train(X,y)

(98, 1)
CASE 1 WITH SIZE_ONLY FEATURE
Model               MAE       MSE       R2        
Random Forest       9.62      204.97    0.67      
XGBoost             9.22      207.55    0.65      
Lasso Regression    10.59     215.20    0.61      


In [70]:
# Case 6 SELECTED FEATURES ONLY - selected xx features
# # With features mentioned in the draft paper
df=df_all_features
pbsq_columns = [col for col in df.columns if 'pbsq' in col]
pbnc_columns = [col for col in df.columns if 'pbnc' in col]
psgc_columns = [col for col in df.columns if 'psgc' in col]
psgcp_columns = [col for col in df.columns if 'psgcp' in col]
pss_columns = [col for col in df.columns if 'pss' in col]
sdl_columns = [col for col in df.columns if 'sdl' in col]
df['sum_pbnc'] = df[pbnc_columns].sum(axis=1)
df['sum_pbsq'] = df[pbsq_columns].sum(axis=1)
df['sum_psgc'] = df[psgc_columns].sum(axis=1)
df['sum_psgcp'] = df[psgcp_columns].sum(axis=1)
df['sum_pss'] = df[pss_columns].sum(axis=1)
df['sdl'] = df[sdl_columns].sum(axis=1)
#remove all columns from df where prefix is pbsq
df = df.loc[:,~df.columns.str.startswith('pbsq')]
df = df.loc[:,~df.columns.str.startswith('pbnc')]
df=df.loc[:,~df.columns.str.startswith('psgc')]
df=df.loc[:,~df.columns.str.startswith('psgcp')]
df=df.loc[:,~df.columns.str.startswith('pss')]
df=df.loc[:,~df.columns.str.startswith('sdl')]
df.columns
df_2 = df.loc[:,['spots','bases','avgLength','size_MB','InsertSize','None_mqc-generalstats-fastqc-percent_duplicates',
                 'None_mqc-generalstats-fastqc-percent_gc','None_mqc-generalstats-fastqc-avg_sequence_length',
                 'None_mqc-generalstats-fastqc-total_sequences','Unique Reads','fsch_Per Tile Sequence Quality',
                 'fsch_Per Sequence Quality Scores','fsch_Per Base Sequence Content','fsch_Per Sequence GC Content',
                 'fsch_Per Base N Content', 'fsch_Overrepresented Sequences','fsch_Per Base Sequence Quality',
                 'sum_pbnc','sum_pbsq','sum_pss','sum_psgc'
                 ]]
df_2['Label'] = y
X = df_2
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
print("Case 6 SELECTED FEATURES ONLY")
train(X,y)

Case 6 SELECTED FEATURES ONLY
Model               MAE       MSE       R2        
Random Forest       7.83      153.65    0.71      
XGBoost             8.02      177.96    0.68      
Lasso Regression    9.31      191.58    0.65      
